# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")

#limit number of pulls so that I dont run out of funds on API
# cities_df = cities_df.head(20)
cities_df

,Unnamed: 0,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cherskiy,68.75,161.30,-4.72,86,100,4.45,RU,1607562405
1,1,qaanaaq,77.48,-69.36,3.27,88,100,3.53,GL,1607562406
2,2,chokurdakh,70.63,147.92,-21.06,88,100,3.49,RU,1607562406
3,3,ushuaia,-54.80,-68.30,44.60,93,90,6.93,AR,1607562406
4,4,dikson,73.51,80.55,-11.65,86,2,2.62,RU,1607562406
...,...,...,...,...,...,...,...,...,...,...
549,549,miyako,39.64,141.95,45.97,57,100,7.54,JP,1607562486
550,550,norheimsund,60.37,6.15,42.01,60,0,18.34,NO,1607562486
551,551,farafangana,-22.82,47.83,73.08,87,98,8.34,MG,1607562486
552,552,nago,26.59,127.98,69.80,94,75,5.82,JP,1607562300


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

#create variables to store location and humidity for Heatmap
locations = cities_df[['Lat','Lng']].astype(float)

humidity = cities_df['Humidity'].astype(float)


In [4]:
# Create a Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#create drop statements for <80, >70 Temp, 0 cloudiness and wind speed <10

#find temp greater than 70 but less than 80
vacation_cities = cities_df[cities_df['Max temp']>70]
vacation_cities = vacation_cities[vacation_cities['Max temp']<80]
#find cities with no clouds
vacation_cities = vacation_cities[vacation_cities['Cloudiness']==0]
#find cities with less than 10mph wind
vacation_cities = vacation_cities[vacation_cities['Wind Speed']<10]


vacation_cities

,Unnamed: 0,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
81,81,sur,22.57,59.53,70.11,72,0,5.44,OM,1607562416
90,90,am timan,11.03,20.28,70.93,23,0,4.50,TD,1607562418
96,96,mecca,21.43,39.83,71.80,49,0,0.94,SA,1607562418
118,118,bonthe,7.53,-12.51,79.20,85,0,5.88,SL,1607562422
178,178,saint-philippe,-21.36,55.77,73.40,78,0,8.05,RE,1607562431
183,183,ampanihy,-24.70,44.75,73.11,68,0,6.26,MG,1607562432
203,203,lazaro cardenas,17.96,-102.20,78.15,80,0,6.02,MX,1607562434
284,284,dzaoudzi,-12.79,45.27,77.00,88,0,3.36,YT,1607562282
299,299,boueni,-12.90,45.08,77.00,88,0,3.36,YT,1607562449
328,328,nueva helvecia,-34.30,-57.23,75.00,63,0,6.78,UY,1607562173


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_cities
#create an empty Hotel name column in new dataframe
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
81,81,sur,22.57,59.53,70.11,72,0,5.44,OM,1607562416,
90,90,am timan,11.03,20.28,70.93,23,0,4.50,TD,1607562418,
96,96,mecca,21.43,39.83,71.80,49,0,0.94,SA,1607562418,
118,118,bonthe,7.53,-12.51,79.20,85,0,5.88,SL,1607562422,
178,178,saint-philippe,-21.36,55.77,73.40,78,0,8.05,RE,1607562431,
183,183,ampanihy,-24.70,44.75,73.11,68,0,6.26,MG,1607562432,
203,203,lazaro cardenas,17.96,-102.20,78.15,80,0,6.02,MX,1607562434,
284,284,dzaoudzi,-12.79,45.27,77.00,88,0,3.36,YT,1607562282,
299,299,boueni,-12.90,45.08,77.00,88,0,3.36,YT,1607562449,
328,328,nueva helvecia,-34.30,-57.23,75.00,63,0,6.78,UY,1607562173,


In [7]:
#geocoordinates
# target_coordinates = hotel_df[['Lat','Lng']]
target_radius = 5000
target_type = "hotel"

#set params dictonary

params = {
         "radius": target_radius,
         "type": target_type,
         "key":g_key}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, location in hotel_df.iterrows():
    #get lat and long from database
    target_coordinates = f"{location['Lat']},{location['Lng']}"
    #append results to params
    params['location'] = target_coordinates
#     print(params)
    response = requests.get(base_url, params=params)

    places_data = response.json()
    hotel_df.loc[index, 'Hotel Name'] = places_data['results'][1]['name']
# Print the json (pretty printed)
# print(json.dumps(places_data, indent=4, sort_keys=True))
hotel_df

,Unnamed: 0,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
81,81,sur,22.57,59.53,70.11,72,0,5.44,OM,1607562416,Sur Plaza Hotel
90,90,am timan,11.03,20.28,70.93,23,0,4.50,TD,1607562418,Yetibey Ngarsira
96,96,mecca,21.43,39.83,71.80,49,0,0.94,SA,1607562418,Hotel Fairmont Makkah Clock Royal Tower
118,118,bonthe,7.53,-12.51,79.20,85,0,5.88,SL,1607562422,Bonthe Holiday Village
178,178,saint-philippe,-21.36,55.77,73.40,78,0,8.05,RE,1607562431,"Chambres d'hôte ""La Trinité"""
183,183,ampanihy,-24.70,44.75,73.11,68,0,6.26,MG,1607562432,Ampanihy - Ouest Hospital
203,203,lazaro cardenas,17.96,-102.20,78.15,80,0,6.02,MX,1607562434,Collection O Hotel Quinta Antigua
284,284,dzaoudzi,-12.79,45.27,77.00,88,0,3.36,YT,1607562282,Multiauto Mayotte Airport
299,299,boueni,-12.90,45.08,77.00,88,0,3.36,YT,1607562449,abalone plongée
328,328,nueva helvecia,-34.30,-57.23,75.00,63,0,6.78,UY,1607562173,Banco República


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
# Create bank symbol layer
hotel_layer = gmaps.marker_layer(
    locations, info_box_content=hotel_info)

# Display figure
fig = gmaps.figure(center=hotel_df[["Lat", "Lng"]].iloc[0], zoom_level = 1.6)

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))